In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
climate_type_dict = {'1':'BSk',
 '9': 'Dfb',
 '4': 'Cfa',
 '7': 'Csb',
 '8': 'Dfa',
 '3': 'BWk',
 '10': 'Dfc',
 '2': 'BWh',
 '6': 'Csa',
 '11': 'Dsb',
 '0': 'BSh',
 '5': 'Cfb',
 '12': 'Dsc',
 '13': 'Dwa',
 '14': 'Dwb'}

In [4]:
all_cols = add_train.columns.tolist()

In [5]:
target=["contest-tmp2m-14d__tmp2m"]

In [6]:
train_df =  pd.read_csv('fe_v1_train.csv')
test_df  =  pd.read_csv('fe_v1_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

climateregions__climateregion


In [7]:
exclude_cols = ['index', 'startdate']
best_cols = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(best_cols))

Main features: 63


In [8]:
for c in all_cols:
    if 'contest-precip-14d' in c:
        print(c)

contest-precip-14d__precip


In [9]:
def calculate_season(month):
    if month == 12 or month == 1 or month == 2:
        return 4
    if month == 9 or month == 10 or month == 11:
        return 3
    if month == 7 or month == 8 or month == 6:
        return 2
    if month == 3 or month == 4 or month == 5:
        return 1

In [10]:
train_df['month'] = add_train['month']
test_df['month'] = add_test['month']

train_df['season'] = train_df['month'].apply(calculate_season)
test_df['season'] = test_df['month'].apply(calculate_season)

train_df['loc_group'] = add_train['loc_group']
test_df['loc_group'] = add_test['loc_group']

train_df['year'] = add_train['year']
test_df['year'] = add_test['year']

def map_climate_type(x):
    return climate_type_dict[str(x)]

train_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)
test_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)

precip_col = 'contest-precip-14d__precip'
wind_col = 'contest-wind-h500-14d__wind-hgt-500'

group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']


In [11]:
slp_col = 'contest-slp-14d__slp'

In [12]:
print(train_df[slp_col].min(), train_df[slp_col].max())
print(test_df[slp_col].min(), test_df[slp_col].max())

100379.21 103275.84
100593.22 102258.27


In [13]:
train_df[slp_col] = train_df[slp_col] - 100000
test_df[slp_col] = test_df[slp_col] - 100000

In [14]:
check_cols = [c for c in train_df.columns if c not in target and c not in exclude_cols]

df = pd.concat([train_df[check_cols], test_df[check_cols]], axis=0)
df.shape

(407088, 68)

In [15]:
def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

In [16]:
def assign_train_test_df(my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()

In [17]:
first_slp_df = create_df_by_group_and_agg(df, season_group_cols, "first", slp_col, "first_slp")
first_slp_df.head(3)

Nan


,loc_group,climate_type,year,season,first_slp
0,0,BSh,2014,3,1352.08
1,0,BSh,2014,4,2072.81
2,0,BSh,2015,1,1917.24


In [18]:
df = df.merge(first_slp_df, how='left', on=season_group_cols)
df['diff_slp_first'] = df[slp_col] - df['first_slp']
assign_train_test_df('diff_slp_first')

In [19]:
features = best_cols + [
    'diff_slp_first'
]

In [20]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=10, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(10)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/10
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = s
df.to_csv('submission_slp_v1.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

10it [00:10,  1.07s/it]


(375734, 64) (375734,)


  0%|                                                                                               | 0/10 [00:00<?, ?it/s]

Learning rate set to 0.073629
0:	learn: 9.2559190	test: 9.1895026	best: 9.1895026 (0)	total: 11.8ms	remaining: 59.1s
200:	learn: 1.1652037	test: 1.1650151	best: 1.1650151 (200)	total: 1.16s	remaining: 27.8s
400:	learn: 0.9089806	test: 0.9107516	best: 0.9107516 (400)	total: 2.15s	remaining: 24.7s
600:	learn: 0.7797909	test: 0.7846072	best: 0.7846072 (600)	total: 3.32s	remaining: 24.3s
800:	learn: 0.6960305	test: 0.7037020	best: 0.7037020 (800)	total: 4.46s	remaining: 23.4s
1000:	learn: 0.6355540	test: 0.6456774	best: 0.6456774 (1000)	total: 5.51s	remaining: 22s
1200:	learn: 0.5912868	test: 0.6030987	best: 0.6030987 (1200)	total: 6.69s	remaining: 21.2s
1400:	learn: 0.5573031	test: 0.5704500	best: 0.5704500 (1400)	total: 7.65s	remaining: 19.7s
1600:	learn: 0.5282239	test: 0.5423669	best: 0.5423669 (1600)	total: 8.58s	remaining: 18.2s
1800:	learn: 0.5042303	test: 0.5192689	best: 0.5192689 (1800)	total: 9.51s	remaining: 16.9s
2000:	learn: 0.4828347	test: 0.4991104	best: 0.4991104 (2000)	tot

 10%|████████▋                                                                              | 1/10 [00:33<05:03, 33.70s/it]

...

Learning rate set to 0.073629
0:	learn: 9.2495159	test: 9.2538532	best: 9.2538532 (0)	total: 5.54ms	remaining: 27.7s
200:	learn: 1.1794871	test: 1.1896459	best: 1.1896459 (200)	total: 938ms	remaining: 22.4s
400:	learn: 0.9146083	test: 0.9241842	best: 0.9241842 (400)	total: 1.84s	remaining: 21.1s
600:	learn: 0.7852367	test: 0.7950755	best: 0.7950755 (600)	total: 2.79s	remaining: 20.4s
800:	learn: 0.6986290	test: 0.7098042	best: 0.7098042 (800)	total: 3.74s	remaining: 19.6s
1000:	learn: 0.6403208	test: 0.6517476	best: 0.6517476 (1000)	total: 4.63s	remaining: 18.5s
1200:	learn: 0.5966201	test: 0.6097004	best: 0.6097004 (1200)	total: 5.51s	remaining: 17.4s
1400:	learn: 0.5593153	test: 0.5735533	best: 0.5735533 (1400)	total: 6.45s	remaining: 16.6s
1600:	learn: 0.5298614	test: 0.5451194	best: 0.5451194 (1600)	total: 7.36s	remaining: 15.6s
1800:	learn: 0.5062549	test: 0.5222190	best: 0.5222190 (1800)	total: 8.43s	remaining: 15s
2000:	learn: 0.4848882	test: 0.5018915	best: 0.5018915 (2000)	tot

 90%|██████████████████████████████████████████████████████████████████████████████▎        | 9/10 [04:56<00:32, 32.69s/it]

Learning rate set to 0.073629
0:	learn: 9.2538744	test: 9.2035439	best: 9.2035439 (0)	total: 6.42ms	remaining: 32.1s
200:	learn: 1.1654628	test: 1.1635299	best: 1.1635299 (200)	total: 915ms	remaining: 21.8s
400:	learn: 0.9127642	test: 0.9162782	best: 0.9162782 (400)	total: 1.87s	remaining: 21.5s
600:	learn: 0.7867641	test: 0.7923857	best: 0.7923857 (600)	total: 2.84s	remaining: 20.8s
800:	learn: 0.7026553	test: 0.7093101	best: 0.7093101 (800)	total: 3.72s	remaining: 19.5s
1000:	learn: 0.6429046	test: 0.6510273	best: 0.6510273 (1000)	total: 4.65s	remaining: 18.6s
1200:	learn: 0.5995640	test: 0.6089357	best: 0.6089357 (1200)	total: 5.61s	remaining: 17.7s
1400:	learn: 0.5641146	test: 0.5743703	best: 0.5743703 (1400)	total: 6.5s	remaining: 16.7s
1600:	learn: 0.5357585	test: 0.5471772	best: 0.5471772 (1600)	total: 7.39s	remaining: 15.7s
1800:	learn: 0.5094200	test: 0.5219097	best: 0.5219097 (1800)	total: 8.33s	remaining: 14.8s
2000:	learn: 0.4881414	test: 0.5015184	best: 0.5015184 (2000)	to

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 27.29it/s]


cos_sim with best submission: 0.9998581196819666
